In [19]:
from getdata import GetDataset
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

In [20]:
data,target = GetDataset(CreateDataFrame=True).Outputdata()

In [21]:
x = np.linspace(data[0][0].min(),data[0][0].max(),900)
data2 = np.zeros((data.shape[0],900))
for i in range(data.shape[0]):
    data2[i,:] = np.interp(x,data[i][0][1:],data[i][1][1:])



In [26]:
np.sqrt(901)
dim = int(np.sqrt(data2.shape[1]))
DataC = np.zeros((data2.shape[0],dim,dim,2))

for i in range(DataC.shape[0]):
    DataC[i,:,:,0] = data2[i].reshape((1,dim,dim))
    DataC[i,:,:,1] = np.gradient(data2[i]).reshape((1,dim,dim))

In [86]:
import tensorflow as tf
from keras import Model, layers, regularizers, losses
from functools import partial

DefaultConv2D = partial(layers.Conv2D,
                        kernel_size=(1,5), activation='sigmoide', padding="same")

DefaultPooling = partial(layers.MaxPool2D,
                         pool_size=2)

def fcn_baseline(input_shape=(30, 30, 2), name='FCN'):
    # Encoder
    input_ = layers.Input(shape=input_shape)

    x = DefaultConv2D(
        32, name='Conv10')(input_)
    x = DefaultConv2D(
        32,name='Conv11')(x)

    x = DefaultPooling(name='Pool10')(x)  # 30x30x2 -> 15x15x32


    x = DefaultConv2D(
        32,  name='Conv20')(x)
    x = DefaultConv2D(
        32,  name='Conv21')(x)

    x = DefaultPooling(name='Pool20')(x)  # 15x15x32 -> 7x7x32

    x = DefaultConv2D(
        64, name='Conv30')(x)
    x = DefaultConv2D(
        64,  name='Conv31')(x)

    x  = DefaultPooling(name='Pool30')(x)  # 7x7x32-> 3x3x64

    x = layers.Flatten(name='Flatten10')(x)
    x =  layers.Dense(288,activation='sigmoide',input_shape=x.shape,name='Dense10')(x)
    x =  layers.Dense(144,activation='sigmoide',input_shape=(x.shape,),name='Dense20')(x)
    x =  layers.Dense(48,activation='sigmoide',input_shape=(x.shape,),name='Dense30')(x)
    x =  layers.Dense(24,activation='sigmoide',input_shape=(x.shape,),name='Dense40')(x)
    x =  layers.Dense(1,activation='relu',input_shape=(x.shape,),name='Dense50')(x)
    modelo = Model(input_,x,name=name)
    modelo.compile(loss=losses.MeanSquaredError(), optimizer='adam', metrics=['mean_squared_error','mean_absolute_error'])
    return modelo


model = fcn_baseline()
model.summary()


Model: "FCN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_32 (InputLayer)       [(None, 30, 30, 2)]       0         
                                                                 
 Conv10 (Conv2D)             (None, 30, 30, 32)        352       
                                                                 
 Conv11 (Conv2D)             (None, 30, 30, 32)        5152      
                                                                 
 Pool10 (MaxPooling2D)       (None, 15, 15, 32)        0         
                                                                 
 Conv20 (Conv2D)             (None, 15, 15, 32)        5152      
                                                                 
 Conv21 (Conv2D)             (None, 15, 15, 32)        5152      
                                                                 
 Pool20 (MaxPooling2D)       (None, 7, 7, 32)          0       

In [83]:
from sklearn.model_selection import train_test_split
target = target.astype(np.float32)
DataC = DataC.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(DataC, target, test_size=0.2, random_state=42, shuffle=True)


In [87]:
model.fit(X_train,y_train,batch_size=64,epochs=10)

Epoch 1/10
39/39 [==============================] - 5s 89ms/step - loss: 9.6254 - mean_squared_error: 9.6254 - mean_absolute_error: 2.7052
Epoch 2/10
39/39 [==============================] - 3s 89ms/step - loss: 7.4262 - mean_squared_error: 7.4262 - mean_absolute_error: 2.6447
Epoch 3/10
39/39 [==============================] - 3s 86ms/step - loss: 7.4262 - mean_squared_error: 7.4262 - mean_absolute_error: 2.6447
Epoch 4/10
39/39 [==============================] - 4s 101ms/step - loss: 7.4262 - mean_squared_error: 7.4262 - mean_absolute_error: 2.6447
Epoch 5/10
39/39 [==============================] - 3s 83ms/step - loss: 7.4262 - mean_squared_error: 7.4262 - mean_absolute_error: 2.6447
Epoch 6/10
39/39 [==============================] - 3s 75ms/step - loss: 7.4262 - mean_squared_error: 7.4262 - mean_absolute_error: 2.6447
Epoch 7/10
39/39 [==============================] - 3s 80ms/step - loss: 7.4262 - mean_squared_error: 7.4262 - mean_absolute_error: 2.6447
Epoch 8/10
39/39 [========